In [1]:
import json
import csv
import pandas as pd
import re
import os
import tqdm

## 上場企業リスト中にnanがある
- 株式会社ピックルスホールディングス

In [2]:
company = pd.read_csv("TDNet_full_data/company_list.csv",encoding="shift-jis",index_col=0)

In [3]:
company

,ＥＤＩＮＥＴコード,提出者種別,上場区分,連結の有無,資本金,決算日,提出者名,提出者法人番,証券コード,提出者業種,所在地,提出者名（ヨミ）
0,E00004,内国法人・組合,上場,有,1491.0,5月31日,カネコ種苗株式会社,5.070000e+12,13760.0,水産・農林業,前橋市古市町一丁目５０番地１２,カネコシュビョウカブシキガイシャ
1,E00006,内国法人・組合,上場,有,13500.0,5月31日,株式会社サカタのタネ,6.020000e+12,13770.0,水産・農林業,横浜市都筑区仲町台２−７−１,カブシキガイシャサカタノタネ
2,E00007,内国法人・組合,上場,有,100.0,3月31日,株式会社雪国まいたけ,1.010000e+12,13750.0,水産・農林業,南魚沼市余川８９番地,カブシキカイシャユキグニマイタケ
3,E00008,内国法人・組合,上場,有,5500.0,3月31日,ホクト株式会社,6.100000e+12,13790.0,水産・農林業,長野市南堀１３８番地１,ホクトカブシキガイシャ
4,E00009,内国法人・組合,上場,有,452.0,6月30日,株式会社アクシーズ,5.340000e+12,13810.0,水産・農林業,鹿児島市草牟田二丁目１番８号,カブシキガイシャアクシーズ
...,...,...,...,...,...,...,...,...,...,...,...,...
10133,E37788,内国法人・組合,非上場,有,3000.0,3月31日,日本管財ホールディングス株式会社,NaN,NaN,サービス業,西宮市六湛寺町９番１６号,ニホンカンザイホールディングスカブシキガイシャ
10134,E37791,内国法人・組合（有価証券報告書等の提出義務者以外）,NaN,NaN,1.0,NaN,株式会社サンワ,9.200000e+12,NaN,内国法人・組合（有価証券報告書等の提出義務者以外）,関市千疋７０５番地,カブシキカイシャサンワ
10135,E37802,内国法人・組合（有価証券報告書等の提出義務者以外）,NaN,NaN,1.0,NaN,株式会社トワ・スール,9.010000e+12,NaN,内国法人・組合（有価証券報告書等の提出義務者以外）,世田谷区羽根木１丁目２９番１８号,カブシキガイシャトワスール
10136,E37806,外国法人・組合（有価証券報告書等の提出義務者以外）,NaN,NaN,0.0,NaN,ＧＷ＆ＫＩＮＶＥＳＴＭＥＮＴＭＡＮＡＧＥＭＥＮＴ，ＬＬＣ,NaN,NaN,外国法人・組合（有価証券報告書等の提出義務者以外）,アメリカ合衆国マサチューセッツ州０２１１６、ボストン、バークレー・ストリート２２２、１５階,ジーダブルアンドケーインベストメントマネジメントエルエルシー


In [4]:
#上場企業リスト取得
#証券コードのlistを作成
com_list=[]
for i in range(0,len(company)):
    if company["上場区分"][i]=="上場":
        com_list.append(company["証券コード"][i])
    #if len(com_list)==3882:
        #print(i)

In [5]:
len(com_list)

3882

In [6]:
#最初に書いてある，証券コードが記入されていない企業が最後に存在するため削除
com_list = com_list[:-1]

In [7]:
len(com_list)

3881

In [8]:
int(com_list[0])

13760

- 301 通期，連結，日本基準
- 304　1期，連結，日本基準
- 307 2期，連結，日本基準
- 310 3通期，連結，日本基準
- 313 その他四半期，連結，日本基準
- 350　通期の修正
- 352　2期の修正
- 354　その他期の修正
- 356　1期の修正
- 358 　3期の修正
- 360 配当予想の修正

In [93]:
#indexを開く
json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[0])), 'r')
js = json.load(json_open)
#使用するコードのllistを作成
pdf_code_list = []
for i in range(0,len(js["publiclyList"])):
    for j in range(0,len(js["publiclyList"][i]["disclosureItems"])):
        if (js["publiclyList"][i]["disclosureItems"][j][2:] == "301" or 
            js["publiclyList"][i]["disclosureItems"][j][2:] == "304" or 
            js["publiclyList"][i]["disclosureItems"][j][2:] == "307" or
            js["publiclyList"][i]["disclosureItems"][j][2:] == "310" or
            js["publiclyList"][i]["disclosureItems"][j][2:] == "313" or
            js["publiclyList"][i]["disclosureItems"][j][2:] == "350" or
            js["publiclyList"][i]["disclosureItems"][j][2:] == "352" or
            js["publiclyList"][i]["disclosureItems"][j][2:] == "354" or
            js["publiclyList"][i]["disclosureItems"][j][2:] == "356" or
            js["publiclyList"][i]["disclosureItems"][j][2:] == "358" or
            js["publiclyList"][i]["disclosureItems"][j][2:] == "360"):
            pdf_code_list.append(js["publiclyList"][i]["disclosureNumber"])
            break

In [94]:
create_pdf_list(com_list[0])

['20220328512394',
 '20211224560318',
 '20210928403538',
 '20210708463658',
 '20210708463684',
 '20210331488354',
 '20201224438982',
 '20201001498500',
 '20200713461156',
 '20200327485606',
 '20191219439108',
 '20190919499244',
 '20190624460038',
 '20190307487203',
 '20181205445580',
 '20180905403871',
 '20180613463090',
 '20180301480984',
 '20171201428656',
 '20170901466541',
 '20170615407199']

In [8]:
def normalize(sentence):
    sentense = unicodedata.normalize('NFKC' , sentence)
    sentence = zenhan.h2z(sentense)
    sentence = sentence.replace(u' ', u'').replace(u'　', u'' )
    sentence = re.sub("①\n\n","1",sentence)
    #sentence = sentence.replace(u'\n', u'')
    return re.sub("\s*", u'', sentence)

In [9]:
import subprocess
import zenhan
import os
import sys
import re
import unicodedata
import argparse
def create_text(out,file):
    #個別
    #out='file/out/company2/rakuten.txt'
    #file = 'file/pdf/company2/rakuten.pdf'
    line_1 = ""
    text=""
    sentences_list=[]
    #print(file)
    #print(out)
    with open(out,'w') as out:
        mat_eol = re.compile("\n".encode('utf-8'))
        #print(fp)
        sentences = []
        sentence=[]
        p = subprocess.run(('pdftotext', file, '-'), stdout = subprocess.PIPE, stderr = subprocess.PIPE)
        temp = u""
        #print(p.stdout.decode('utf-8'))
        for line in mat_eol.split(p.stdout):
            try:
                line = line.decode('utf-8')
            except:
                line = ""
            if line == "":
                if temp != "" and u"。" in temp:
                    sentences = sentences + re.split(u"。", temp)
                temp = u""
            else:
                temp = temp+line

        if temp != "" and u"。" in temp:
            sentences = sentences + re.split(u"。", temp)
            #print(sentences)
        #print(sentences)
        #sentences = [normalize(line + u"。") for line in sentences if len(line) <= 300 and len(line) >+ 3]
        for line in sentences :
            text=line
            if "①" in line_1 or "②" in line_1 or "③" in line_1 or "④" in line_1 or "⑤" in line_1:
                #print(line_1)
                text = "0" + line
            if len(text) <= 300 and len(text) >+ 3:
                sentences_list.append(normalize(text + u"。"))
                #print(text)
            line_1 = line
            #print(line_1)

        for sentence in sentences_list:
            #print(sentence)
            out.write(sentence+"\n")

In [53]:
for i in tqdm.tqdm(range(0,len(com_list))):
    #indexを開く
    json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[i])), 'r')
    js = json.load(json_open)
   # print(js)
    #break
    #使用するコードのllistを作成，合致するコードが一つでもあれば取得
    pdf_code_list = []
    for i in range(0,len(js["publiclyList"])):
        for j in range(0,len(js["publiclyList"][i]["disclosureItems"])):
            if (js["publiclyList"][i]["disclosureItems"][j][2:] == "301" or 
                js["publiclyList"][i]["disclosureItems"][j][2:] == "304" or 
                js["publiclyList"][i]["disclosureItems"][j][2:] == "307" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "310" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "313" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "350" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "352" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "354" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "356" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "358" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "360"):
                pdf_code_list.append(js["publiclyList"][i]["disclosureNumber"])
                break
    
    for i in range(0,len(pdf_code_list)): 
        out='text_data/{}/{}.txt'.format(js["publiclyList"][0]["code"],pdf_code_list[i])
        file = 'TDNet_full_data/API_data/{}_g/{}_{}.pdf'.format(js["publiclyList"][0]["code"],js["publiclyList"][0]["code"]
                                                                ,pdf_code_list[i])
        dic = 'text_data/{}'.format(js["publiclyList"][0]["code"])
        
        #フォルダを作成する
        try:
            os.makedirs(dic)
        except FileExistsError:
            pass
        ###
        
        #txtを作成
        create_text(out,file)

100%|█████████████████████████████████████| 3881/3881 [1:24:04<00:00,  1.30s/it]


# 途中で日本基準から変わった企業の検索および途中から上場した企業

In [4]:
company = pd.read_csv("TDNet_full_data/company_list.csv",encoding="shift-jis",index_col=0)

In [5]:
#上場企業リスト取得
#証券コードのlistを作成
com_list=[]
for i in range(0,len(company)):
    if company["上場区分"][i]=="上場":
        com_list.append(company["証券コード"][i])
    #if len(com_list)==3882:
        #print(i)
#最初に書いてある，証券コードが記入されていない企業が最後に存在するため削除
com_list = com_list[:-1]

In [7]:
pdf_code_list =[[]*i for i in range(0,len(com_list))]
date_list =[[]*i for i in range(0,len(com_list))]
code_list=[[]*i for i in range(0,len(com_list))]

In [8]:
for com in tqdm.tqdm(range(0,len(com_list))):
    #indexを開く
    json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[com])), 'r')
    js = json.load(json_open)
   # print(js)
    #break
    #使用するコードのllistを作成，合致するコードが一つでもあれば取得
    for i in range(0,len(js["publiclyList"])):
        for j in range(0,len(js["publiclyList"][i]["disclosureItems"])):
            if (js["publiclyList"][i]["disclosureItems"][j][2:] == "301" or 
                js["publiclyList"][i]["disclosureItems"][j][2:] == "304" or 
                js["publiclyList"][i]["disclosureItems"][j][2:] == "307" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "310" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "313"):# or
                #js["publiclyList"][i]["disclosureItems"][j][2:] == "350" or
                #js["publiclyList"][i]["disclosureItems"][j][2:] == "352" or
                #js["publiclyList"][i]["disclosureItems"][j][2:] == "354" or
                #js["publiclyList"][i]["disclosureItems"][j][2:] == "356" or
                #js["publiclyList"][i]["disclosureItems"][j][2:] == "358" or
                #js["publiclyList"][i]["disclosureItems"][j][2:] == "360"):
                pdf_code_list[com].append(js["publiclyList"][i]["disclosureNumber"])
                date_list[com].append(js["publiclyList"][i]["disclosedDate"])
                code_list[com].append(js["publiclyList"][i]["disclosureItems"])
                break


100%|█████████████████████████████████████| 3881/3881 [00:02<00:00, 1601.18it/s]


In [10]:
date_list

[['2022-04-04',
  '2022-01-05',
  '2021-10-06',
  '2021-07-13',
  '2021-04-02',
  '2021-01-05',
  '2020-10-02',
  '2020-07-14',
  '2020-04-02',
  '2020-01-06',
  '2019-10-02',
  '2019-07-12',
  '2019-04-04',
  '2019-01-07',
  '2018-10-02',
  '2018-07-13',
  '2018-04-05',
  '2018-01-05',
  '2017-10-03',
  '2017-07-11'],
 ['2022-04-08',
  '2022-01-13',
  '2021-10-13',
  '2021-07-14',
  '2021-04-09',
  '2021-01-13',
  '2020-10-09',
  '2020-07-13',
  '2020-04-03',
  '2020-01-09',
  '2019-10-04',
  '2019-07-10',
  '2019-04-04',
  '2019-01-11',
  '2018-10-05',
  '2018-07-11',
  '2018-04-06',
  '2018-01-12',
  '2017-10-06',
  '2017-07-13'],
 [],
 ['2022-05-13',
  '2022-02-10',
  '2021-11-12',
  '2021-08-12',
  '2021-05-14',
  '2021-02-02',
  '2020-11-02',
  '2020-08-03',
  '2020-05-15',
  '2020-02-03',
  '2019-11-05',
  '2019-08-02',
  '2019-05-10',
  '2019-02-04',
  '2018-11-02',
  '2018-08-02',
  '2018-05-11',
  '2018-02-02',
  '2017-11-02',
  '2017-08-02'],
 ['2022-04-22',
  '2022-01-28',


In [28]:
not_list=[]
for cnt,d in enumerate(date_list):
    if d!=[]:
        if '2022' not in d[0] and '2017' not in d[-1]:
            not_list.append(cnt)

In [62]:
com=not_list[14]
#indexを開く
json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[com])), 'r')
js = json.load(json_open)

In [63]:
js

{'statusCode': '200',
 'message': None,
 'count': '60',
 'publiclyList': [{'code': '46910',
   'name': 'ワシントンホテル',
   'disclosedDate': '2022-05-27',
   'disclosedTime': '17:00:00',
   'handlingType': None,
   'disclosureNumber': '20220527560772',
   'modifiedHistory': '1',
   'title': '独立役員届出書',
   'disclosureItems': ['11739'],
   'pdfGeneralFlag': '1',
   'pdfSummaryFlag': '0',
   'xbrlFlag': '0'},
  {'code': '46910',
   'name': 'ワシントンホテル',
   'disclosedDate': '2022-05-24',
   'disclosedTime': '15:30:00',
   'handlingType': None,
   'disclosureNumber': '20220516549540',
   'modifiedHistory': '1',
   'title': '資本金の額の減少及び剰余金の処分に関するお知らせ',
   'disclosureItems': ['11103', '11199'],
   'pdfGeneralFlag': '1',
   'pdfSummaryFlag': '0',
   'xbrlFlag': '0'},
  {'code': '46910',
   'name': 'ワシントンホテル',
   'disclosedDate': '2022-05-24',
   'disclosedTime': '15:30:00',
   'handlingType': None,
   'disclosureNumber': '20220519551615',
   'modifiedHistory': '1',
   'title': '役員の異動に関するお知らせ',
   'discl

In [34]:
len(not_list)

15